# Case Studies
Taking a closer look at specific tracts/neighborhoods as they relate to features and model predictions

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import geopandas as gpd

import sys
sys.path.insert(0, './')
from functions import *
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet("Data/Cleaned/after_kriging.parquet")
features = pd.read_parquet("Data/Cleaned/post_feat_engineering.parquet")
labels = pd.read_parquet("Data/Cleaned/labels_udp.parquet")
preds = pd.read_parquet("Data/Cleaned/future_preds_udp.parquet")

## Baselines
First we look at averages across NYC so we have something to compare to

In [34]:
df.groupby('year')[['percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
        'num_evictions_commercial', 'num_evictions_residential']].mean()

,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
year,,,,,,,,,,,,,,,,,,,,
2010,0.242454,0.139965,0.043005,0.016762,0.449438,0.261382,0.124545,0.259714,0.131002,0.088334,0.171134,55528.931168,1156.765636,36.157028,535086.965243,5.938065,0.000000,266.120218,0.418319,5.535434
2011,0.242674,0.137526,0.043336,0.017122,0.448856,0.261790,0.124914,0.259941,0.132909,0.089879,0.170348,57111.915472,1208.907849,36.231290,540994.115973,6.155269,0.000000,269.280613,0.430089,5.969503
2012,0.243551,0.135903,0.043124,0.017312,0.449617,0.261391,0.126017,0.263949,0.134143,0.090371,0.168931,57716.935201,1247.218521,36.327653,534522.202972,6.540215,0.000000,272.592358,0.442505,6.402725
2013,0.242832,0.134236,0.042204,0.017723,0.447559,0.259503,0.127624,0.263208,0.136295,0.092490,0.167461,58449.423601,1279.950349,36.508370,526466.242248,7.267957,4822.243871,275.941954,0.454796,6.836497
2014,0.246214,0.133881,0.042324,0.018007,0.441837,0.258481,0.130972,0.264097,0.140190,0.095211,0.167109,59330.267990,1311.083148,36.623911,526811.641482,7.753118,6951.583656,279.114069,0.467236,7.270738
2015,0.250815,0.134240,0.043164,0.017944,0.437897,0.256035,0.134386,0.264510,0.143412,0.098101,0.165452,60179.990343,1353.480023,36.761491,561132.415163,8.412043,8548.784516,282.848014,0.479840,7.704377
2016,0.253764,0.135691,0.043299,0.018642,0.436232,0.252976,0.137392,0.266742,0.146280,0.100864,0.162704,62185.511682,1406.594339,36.915523,580909.165138,9.189677,11910.135914,285.629622,0.492971,8.137841
2017,0.255537,0.135560,0.043058,0.018995,0.435236,0.251176,0.140188,0.267264,0.148578,0.103868,0.159853,65067.340476,1456.235998,37.334528,609065.251468,10.030968,14076.883441,289.011500,0.506015,8.575314
2018,0.256467,0.136981,0.043575,0.019341,0.431086,0.250792,0.141076,0.269538,0.151588,0.106136,0.157048,68239.983310,1502.724477,37.654115,646815.258788,10.758710,15095.210323,292.426550,0.540951,8.239237


In [35]:
features.groupby('pred_year')[['percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
        'num_evictions_commercial', 'num_evictions_residential']].mean()

,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
pred_year,,,,,,,,,,,,,,,,,,,,
2020,0.018113,0.006759,0.092007,0.132556,0.054645,0.138879,0.170410,0.063371,0.044026,0.090245,0.024107,0.024688,0.037249,0.007308,0.045150,0.063098,0.318214,0.016976,0.032198,0.077127
2021,0.016786,0.011739,0.088805,0.139968,0.049052,0.125641,0.169037,0.058854,0.045983,0.085219,0.017563,0.027360,0.034968,0.008322,0.045900,0.064160,0.239587,0.015174,-0.019618,0.030259
2022,0.015305,0.018323,0.089434,0.146794,0.038535,0.126863,0.168381,0.049784,0.042426,0.081854,0.013209,0.033533,0.036507,0.007786,0.056679,0.073975,0.340944,0.007047,-0.042693,0.027974
2023,0.015955,0.020917,0.077293,0.136494,0.036942,0.125927,0.159893,0.060919,0.045196,0.076418,0.008756,0.040552,0.038460,0.008167,0.063017,0.084070,0.197187,0.005178,-0.133215,0.011284
2024,0.002244,0.021928,0.070475,0.315161,0.029528,0.119090,0.133687,0.056693,0.044081,0.075320,0.006131,0.045995,0.040661,0.008433,0.072927,0.089004,1.234085,0.003492,-0.264729,-0.127031
2025,-0.013264,0.022502,0.057796,0.414669,0.015978,0.116558,0.132577,0.052062,0.040223,0.072535,0.005295,0.055950,0.045698,0.008995,0.069112,0.087648,5.068263,0.001726,-0.329470,-0.238397
2026,-0.022915,0.021602,0.059805,0.456526,-0.002793,0.115251,0.169778,0.063214,0.039746,0.072725,0.006882,0.065355,0.053235,0.009548,0.088390,0.076699,4.122435,0.000586,-0.005809,0.149221
2027,-0.028600,0.021810,0.066574,0.474557,-0.017343,0.120716,0.143567,0.067232,0.037700,0.076038,0.014038,0.066232,0.054831,0.010085,0.105279,0.078682,42.643693,-0.001285,0.006161,0.406266


## Manhattan - Harlem
- population trend: down = displacement?
- 

In [33]:
df_ny = df[df.GEOID=='36061016600']  # Harlem
df_ny[['year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
1358,2010,0.346733,0.036714,0.043933,0.003609,0.344493,0.201618,0.068699,0.595271,0.103423,0.078158,0.205227,34491.000000,947.0,31.4,9.038000e+05,0.0,0.0,173.0,1.375,3.875
1358,2011,0.359608,0.036793,0.057473,0.017860,0.394924,0.195246,0.078287,0.569760,0.121257,0.092520,0.171747,37174.000000,1022.0,32.1,7.031000e+05,0.0,0.0,187.0,1.750,5.750
1358,2012,0.331620,0.043380,0.056670,0.023947,0.450853,0.183801,0.081494,0.566951,0.127633,0.080241,0.192202,34310.000000,1010.0,32.3,5.938000e+05,0.0,0.0,201.0,2.125,7.625
1358,2013,0.320596,0.045078,0.067746,0.019560,0.494560,0.173834,0.068653,0.553238,0.157642,0.079793,0.179275,35706.000000,1034.0,30.8,3.817000e+05,0.0,0.0,216.0,2.500,9.500
1358,2014,0.347459,0.027038,0.085142,0.021708,0.476017,0.157676,0.049526,0.548291,0.145197,0.077733,0.180944,37959.000000,1149.0,31.8,5.026000e+05,0.0,0.0,230.0,2.875,11.375
1358,2015,0.384247,0.020375,0.085601,0.028032,0.466293,0.149460,0.044305,0.555996,0.158622,0.075756,0.178449,35306.000000,1100.0,32.1,3.750000e+05,0.0,0.0,245.0,3.250,13.250
1358,2016,0.368762,0.017472,0.083918,0.019987,0.516347,0.159762,0.033752,0.511582,0.165189,0.068696,0.141760,37227.000000,1056.0,31.7,3.424000e+05,0.0,5865.0,259.0,3.625,15.125
1358,2017,0.377278,0.016942,0.082432,0.022779,0.516800,0.150484,0.041287,0.545700,0.173121,0.069191,0.140803,40593.000000,1086.0,32.4,1.106936e+06,0.0,17707.0,273.0,4.000,17.000
1358,2018,0.376464,0.014673,0.078850,0.022380,0.510597,0.161998,0.041055,0.552987,0.150585,0.093671,0.138728,44356.000000,1176.0,34.2,1.108539e+06,0.0,19300.0,288.0,2.000,16.000
1358,2019,0.359417,0.022738,0.063509,0.019288,0.554963,0.138153,0.048299,0.574878,0.166066,0.100361,0.103340,40402.000000,1172.0,34.1,6.173000e+05,0.0,25023.0,302.0,0.000,18.000


In [25]:
df_ny_change = features[features.GEOID=='36061016600']  # Harlem
df_ny_change[['year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,GEOID,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,med_hh_age,med_hh_tenure,num_citibike_rides,num_trees,med_rent,num_affordable_hous_built,num_bikelanes,num_evictions_commercial,num_evictions_residential,park_acres,num_parks,pred_year,lag_vulnerable
1358,36061016600,-0.009007,-0.013184,0.035948,0.004834,0.0,0.308274,0.048081,0.295684,0.137135,-0.081438,-0.053766,0.056566,0.061022,0.012572,-0.094800,0.417636,0.122032,0.703354,0.072396,-0.036763,0.215520,-0.099875,0.0,-0.024457,0.085843,-0.016497,0.025483,0.028863,-0.053562,0.001102,-0.091471,0.014842,0.019965,0.001921,-0.252422,-0.126463,0.087440,0.0,-0.001572,-0.014822,0.000000,0.069599,0.024536,0.000000,0.0,0.176550,0.259929,0.0,0.0,2020,0.317460
3683,36061016600,-0.008524,0.019995,0.046096,-0.033329,0.0,0.182854,0.011735,0.073864,0.084231,-0.074386,-0.010052,0.039564,0.102285,0.010232,-0.100214,0.071377,0.067713,0.068584,0.048144,-0.041175,0.221113,-0.085929,0.0,-0.006199,0.065106,-0.045923,0.009942,0.028594,-0.027498,-0.004594,-0.014587,0.016947,0.011500,0.001886,-0.332357,0.282693,0.095590,0.0,0.006203,-0.006886,2.019096,0.065120,0.027619,2.666667,0.0,0.148337,0.199945,0.0,0.0,2021,0.333333
6008,36061016600,-0.026936,0.066640,0.049931,-0.026978,0.0,0.089689,-0.000692,0.067243,0.059528,-0.071645,-0.004026,0.056988,0.101177,0.022843,-0.152369,0.151934,0.062800,0.008860,0.022540,-0.018653,0.331070,-0.093693,0.0,-0.003152,0.034647,0.035164,0.003354,0.065806,-0.049803,-0.006827,0.009556,0.045237,0.027270,0.010106,0.001996,0.308843,0.119506,0.0,-0.160464,-0.014461,1.054530,0.061800,0.027028,0.833333,0.0,0.029290,0.135793,0.0,0.0,2022,0.333333
8333,36061016600,-0.030727,0.092588,0.073275,-0.037607,0.0,0.052045,0.005201,-0.005206,0.064704,-0.021754,0.015535,0.056791,0.064178,0.020837,-0.067287,0.146494,-0.002201,0.016366,0.020865,-0.034148,0.344209,-0.038026,0.0,0.007477,0.012593,0.047997,0.003448,0.077906,-0.081109,0.000550,-0.020403,0.023599,0.022742,0.017359,0.000503,0.294518,0.091886,0.0,-0.185328,-0.023088,0.801863,0.057464,0.026763,0.833333,0.0,-0.166789,0.115643,0.0,0.0,2023,0.333333
10658,36061016600,-0.039945,0.016305,0.044065,-0.046553,0.0,-0.083493,-0.028707,-0.083277,-0.017027,0.019864,0.048893,-0.013394,0.023801,-0.018254,-0.008492,0.119132,-0.001355,0.155319,0.001751,-0.012664,0.326670,0.220049,0.0,0.001412,0.028640,0.043898,-0.009591,0.109411,-0.053437,0.008242,-0.028895,0.043510,0.008330,0.022700,-0.002008,0.263355,0.098824,0.0,-0.193328,-0.059372,0.637634,0.054940,0.016191,0.833333,0.0,-0.230146,-0.083919,0.0,0.0,2024,0.333333
12983,36061016600,-0.038654,0.093364,0.008820,-0.032246,0.0,-0.223060,-0.034072,-0.135959,-0.032668,0.081827,0.022561,-0.047851,0.084739,-0.043466,0.027784,0.156431,-0.018872,0.136423,-0.007739,-0.020221,0.052382,0.274993,0.0,-0.000761,0.004695,0.067239,-0.013761,0.132918,-0.045566,-0.001157,0.060202,0.153236,0.030425,0.023424,-0.006083,0.303804,0.022625,0.0,-0.206383,-0.083181,0.492306,0.051431,0.000562,0.555556,0.0,-0.320292,-0.133669,0.0,0.0,2025,0.333333
15308,36061016600,-0.039418,0.133625,-0.006488,-0.025286,0.0,0.016544,-0.106995,-0.108283,0.030525,0.097400,-0.050668,-0.050948,0.093233,-0.055969,0.012089,0.250388,-0.054187,0.291922,-0.050627,-0.052250,0.006620,0.329799,0.0,-0.001914,-0.015438,0.130397,-0.023680,0.131671,-0.026693,-0.008824,0.180218,0.075539,0.041675,0.021877,-0.008948,0.3454

## Bronx


In [38]:
df_bronx = df[df.GEOID=='36047000200']  # Bronx
df_bronx[['year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
362,2010,0.153846,0.074951,0.226824,0.000000,0.368836,0.116371,0.109467,0.813609,0.024655,0.000000,0.093688,54688.0,700.0,38.7,481500.0,0.0,0.0,97.0,0.0,1.375
362,2011,0.135861,0.063191,0.128752,0.014218,0.485782,0.139021,0.033965,0.819905,0.041074,0.000000,0.151659,31728.0,1021.0,27.8,459100.0,0.0,0.0,107.0,0.0,1.750
362,2012,0.156039,0.013503,0.156789,0.025506,0.360840,0.140285,0.024006,0.798950,0.034509,0.000000,0.225806,33988.0,1241.0,28.3,488600.0,0.0,0.0,116.0,0.0,2.125
362,2013,0.206576,0.019851,0.096774,0.022333,0.312035,0.099876,0.000000,0.859181,0.019851,0.000000,0.335608,33906.0,1228.0,29.2,546300.0,0.0,0.0,126.0,0.0,2.500
362,2014,0.237028,0.027546,0.092249,0.024984,0.207559,0.098014,0.000000,0.856502,0.025625,0.000000,0.350416,35500.0,1286.0,28.0,473700.0,0.0,0.0,136.0,0.0,2.875
362,2015,0.238758,0.028908,0.087259,0.021949,0.247323,0.101713,0.004818,0.869379,0.022484,0.003212,0.360814,34740.0,1344.0,28.7,462500.0,0.0,0.0,146.0,0.0,3.250
362,2016,0.291819,0.043346,0.087912,0.017094,0.208181,0.040293,0.016484,0.897436,0.034188,0.018926,0.380342,48393.0,1327.0,31.1,462100.0,0.0,0.0,155.0,0.0,3.625
362,2017,0.300797,0.057105,0.059097,0.007304,0.312085,0.031873,0.037185,0.887782,0.039841,0.021912,0.381806,34931.0,1235.0,31.2,450000.0,0.0,841.0,165.0,0.0,4.000
362,2018,0.275945,0.082060,0.055511,0.008850,0.451327,0.038616,0.049879,0.838294,0.063556,0.032180,0.252615,39712.0,1241.0,32.2,450000.0,12.0,25.0,175.0,1.0,1.000
362,2019,0.290488,0.090831,0.035990,0.005141,0.556984,0.031705,0.049700,0.778063,0.068552,0.062554,0.206512,56080.0,1497.0,32.0,562500.0,12.0,0.0,184.0,1.0,0.000


In [40]:
df_change_bronx = features[features.GEOID=='36047000200']  # Bronx
df_change_bronx[['pred_year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
362,2020,0.122073,-0.115126,0.089114,-0.062815,-0.111513,0.109576,0.016959,0.128367,4.892145,0.286975,0.011265,0.123876,-0.027390,-0.003596,0.0,0.000000,0.081325,0.000000,0.176550
2687,2021,0.146685,-0.097692,-0.021190,-0.032475,-0.178782,0.595973,0.013877,0.044928,2.524983,0.184490,0.034874,0.035892,0.020088,-0.000206,0.0,0.000000,0.074896,0.000000,0.148337
5012,2022,0.108162,-0.144100,-0.118251,0.084752,-0.145034,0.754627,0.008846,0.170779,1.839518,0.046610,0.045814,0.000790,0.022433,-0.010916,0.0,-0.970273,0.070978,0.000000,-0.012377
7337,2023,0.062967,-0.138914,-0.167349,0.146312,-0.126854,1.003729,-0.015694,0.254671,1.615601,-0.064850,0.114911,0.036916,0.016097,0.011069,0.0,-0.985137,0.065182,0.000000,-0.208455
9662,2024,0.088043,-0.106387,0.430471,0.169982,-0.168533,0.789586,-0.024856,0.265010,1.408339,-0.111955,0.105105,0.050421,0.047947,0.108536,0.0,-0.985137,0.061012,0.000000,-0.280146
11987,2025,0.038298,0.010590,0.533478,0.146510,-0.186567,0.661737,-0.030718,0.282989,1.132026,-0.116370,0.119094,0.039253,0.042421,0.119742,0.0,-0.985137,0.057348,-0.333333,-0.506233
14312,2026,0.020622,0.018376,0.647604,0.121709,0.025016,0.205696,-0.035340,0.197923,0.326772,-0.110562,0.110374,0.082826,0.028027,0.160502,0.0,-0.555352,0.054427,-0.333333,-0.661638
16637,2027,-0.013693,0.107027,0.772471,0.043264,0.039837,-0.008186,-0.034854,0.143282,0.327611,-0.083517,0.161407,0.097531,0.028865,0.149378,0.0,-0.213821,0.051499,-0.125000,-0.916667


In [ ]:
## TODO!!! percent changes aren't registering when the first number is 0 - find a way to address this


## Brooklyn

In [44]:
df_bk = df[df.GEOID=='36047042300']  # BK
df_bk[['year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
746,2010,0.258880,0.072640,0.003840,0.568960,0.069440,0.007360,0.812800,0.022080,0.028800,0.241600,36578.0,1191.0,29.4,617300.0,8.0,0.0,196.0,0.0,9.375
746,2011,0.308504,0.065103,0.002639,0.669795,0.120528,0.008798,0.772434,0.065396,0.038416,0.273021,47868.0,1308.0,30.0,526600.0,8.0,0.0,211.0,0.0,8.750
746,2012,0.267439,0.047572,0.005669,0.647769,0.103771,0.007148,0.819818,0.077644,0.034015,0.279270,37783.0,1298.0,29.6,516100.0,8.0,0.0,226.0,0.0,8.125
746,2013,0.322940,0.051002,0.005568,0.672606,0.148107,0.022494,0.806682,0.071269,0.030067,0.297773,39038.0,1351.0,30.4,541200.0,8.0,0.0,242.0,0.0,7.500
746,2014,0.320346,0.052360,0.024943,0.607916,0.171923,0.050093,0.735312,0.099361,0.036487,0.299114,45648.0,1357.0,29.8,492900.0,8.0,0.0,257.0,0.0,6.875
746,2015,0.343161,0.047934,0.027878,0.595467,0.172684,0.039310,0.739671,0.103490,0.031087,0.334336,51083.0,1375.0,29.0,464500.0,8.0,0.0,273.0,0.0,6.250
746,2016,0.332556,0.038576,0.028826,0.511022,0.167656,0.050869,0.703900,0.104281,0.029674,0.293769,47273.0,1412.0,28.6,483800.0,8.0,0.0,288.0,0.0,5.625
746,2017,0.375456,0.044024,0.030338,0.554060,0.126825,0.052007,0.658075,0.122491,0.036496,0.247491,62750.0,1614.0,28.6,485900.0,8.0,0.0,303.0,0.0,5.000
746,2018,0.365467,0.054003,0.040739,0.513501,0.080057,0.042160,0.608479,0.167219,0.036712,0.243723,67262.0,1690.0,28.2,473200.0,26.0,0.0,319.0,0.0,6.000
746,2019,0.379588,0.066868,0.039718,0.557567,0.064354,0.029663,0.581448,0.183761,0.043238,0.190548,67056.0,2042.0,28.5,847600.0,26.0,22654.0,334.0,1.0,7.000


In [45]:
df_change_bk = features[features.GEOID=='36047042300']  # BK
df_change_bk[['pred_year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
746,2020,0.049732,-0.092346,0.741487,-0.012631,0.193340,0.576737,-0.022541,0.418387,0.020555,0.036686,0.057498,0.029340,-0.004379,-0.037258,0.000,0.000000,0.066267,0.0,-0.081543
3071,2021,0.039284,-0.051514,0.802342,-0.028132,0.030131,0.547910,-0.025115,0.120528,0.003226,-0.011244,0.060622,0.036810,-0.007780,-0.012046,0.000,0.000000,0.062193,0.0,-0.088951
5396,2022,0.057034,0.031145,0.668151,-0.034852,-0.008157,0.547602,-0.047899,0.150174,0.023306,-0.017597,0.107720,0.045932,-0.007889,-0.013079,0.375,0.000000,0.059145,0.0,-0.043713
7721,2023,0.028886,0.058832,0.666954,-0.026940,-0.112056,0.140384,-0.052633,0.180343,0.072277,-0.065002,0.101673,0.073841,-0.010621,0.110683,0.375,0.000000,0.055183,0.0,-0.003114
10046,2024,0.036672,0.005861,0.059098,-0.046489,-0.016189,0.107231,-0.051380,0.098674,0.008546,-0.060218,0.117697,0.097505,-0.000314,0.124358,0.375,0.713296,0.052836,-1.0,-0.132083
12371,2025,0.021079,0.092121,0.160786,-0.040769,-0.004497,0.156978,-0.064340,0.100316,0.183862,-0.086322,0.096070,0.105404,0.009211,0.168443,0.375,0.656573,0.049603,-1.0,-0.283598
14696,2026,-0.003015,0.086048,0.259803,-0.048904,-0.008303,0.135169,-0.045054,0.145045,0.212283,-0.030402,0.131445,0.112196,0.012055,0.152115,0.375,0.521941,0.047295,-1.0,-0.320317
17021,2027,-0.010895,0.003508,0.263086,-0.051771,0.046513,0.128253,-0.044295,0.119326,0.261252,-0.037655,0.108716,0.111363,0.012055,0.131721,0.375,0.439087,0.045632,-1.0,0.301905
